In [1]:
import numpy as np
import pandas as pd 
import re
import os

In [2]:
# Combine insiders' financial activities with whether insider is POI 
def enron_finclean():
    df1 = pd.read_csv('enronfinance.csv')
    df2 = pd.read_csv('enron_crimpros.csv')
    df2 = df2.set_index('insider')
    df1 = df1.set_index('insider')
    df1 = df1.fillna(0)
    df1 = df1.replace('-',0)
    df1 = df1.astype(float)
    df_f = df1.merge(df2,how='left',left_index=True, right_index=True)
    df_f['POI'] = df_f['POI'].fillna('No')
    df_f['bonus_salary_ratio'] = df_f['bonus']/df_f['salary']
    df_f['bonus_salary_ratio'] = df_f['bonus_salary_ratio'].fillna(0)
    return df_f

In [3]:
df = enron_finclean()
df = df.reset_index().rename({'index':'insider'})
df.head()

,insider,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,other,expenses,director_fees,total_payments,stock_op_exercised,restrict_stock,restrict_stock_deferred,total_stock_value,position,POI,bonus_salary_ratio
0,"ALLEN, PHILLIP K",201955.0,4175000.0,304805.0,-3081055.0,2869717.0,0.0,152.0,13868.0,0.0,4484442.0,1729541.0,126027.0,-126027.0,1729541.0,NaN,No,20.672922
1,"BADUM, JAMES P",0.0,0.0,0.0,0.0,178980.0,0.0,0.0,3486.0,0.0,182466.0,257817.0,0.0,0.0,257817.0,NaN,No,0.000000
2,"BANNANTINE, JAMES M",477.0,0.0,0.0,-5104.0,0.0,0.0,864523.0,56301.0,0.0,916197.0,4046157.0,1757552.0,-560222.0,5243487.0,NaN,No,0.000000
3,"BAXTER, JOHN C",267102.0,1200000.0,1586055.0,-1386055.0,1295738.0,0.0,2660303.0,11200.0,0.0,5634343.0,6680544.0,3942714.0,0.0,10623258.0,NaN,No,4.492666
4,"BAY, FRANKLIN R",239671.0,400000.0,0.0,-201641.0,260455.0,0.0,69.0,129142.0,0.0,827696.0,0.0,145796.0,-82782.0,63014.0,NaN,No,1.668955


In [4]:
# Combine insiders' email addresses into the dataset
insider_emails = pd.read_csv('enroninsider_emails.csv')
df = pd.concat([df, insider_emails], axis=1).drop(columns=['name'])
df.head()

,insider,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,other,expenses,director_fees,total_payments,stock_op_exercised,restrict_stock,restrict_stock_deferred,total_stock_value,position,POI,bonus_salary_ratio,email_address
0,"ALLEN, PHILLIP K",201955.0,4175000.0,304805.0,-3081055.0,2869717.0,0.0,152.0,13868.0,0.0,4484442.0,1729541.0,126027.0,-126027.0,1729541.0,NaN,No,20.672922,phillip.allen@enron.com
1,"BADUM, JAMES P",0.0,0.0,0.0,0.0,178980.0,0.0,0.0,3486.0,0.0,182466.0,257817.0,0.0,0.0,257817.0,NaN,No,0.000000,NaN
2,"BANNANTINE, JAMES M",477.0,0.0,0.0,-5104.0,0.0,0.0,864523.0,56301.0,0.0,916197.0,4046157.0,1757552.0,-560222.0,5243487.0,NaN,No,0.000000,james.bannantine@enron.com
3,"BAXTER, JOHN C",267102.0,1200000.0,1586055.0,-1386055.0,1295738.0,0.0,2660303.0,11200.0,0.0,5634343.0,6680544.0,3942714.0,0.0,10623258.0,NaN,No,4.492666,NaN
4,"BAY, FRANKLIN R",239671.0,400000.0,0.0,-201641.0,260455.0,0.0,69.0,129142.0,0.0,827696.0,0.0,145796.0,-82782.0,63014.0,NaN,No,1.668955,frank.bay@enron.com


In [5]:
# Load raw email dataset
emails_raw = pd.read_csv('emails.csv')
print(emails_raw.shape)
emails_raw.head()

(517401, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [6]:
# Example of an email
print(emails_raw.loc[0]['message'])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


In [7]:
# Process raw emails to separate senders' and receivers' email address
def parse_raw_message(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

def parse_into_emails(messages):
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'), 
        'to': map_to_list(emails, 'to'), 
        'from_': map_to_list(emails, 'from')
    }

def map_to_list(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [8]:
email_df = pd.DataFrame(parse_into_emails(emails_raw.message))
email_df = email_df.dropna(how='any', subset=['to', 'from_'])
email_df.head()

,body,to,from_
0,Here is our forecast,tim.belden@enron.com,phillip.allen@enron.com
1,Traveling to have a business meeting takes the...,john.lavorato@enron.com,phillip.allen@enron.com
2,test successful. way to go!!!,leah.arsdall@enron.com,phillip.allen@enron.com
3,"Randy,Can you send me a schedule of the salary...",randall.gay@enron.com,phillip.allen@enron.com
4,,greg.piper@enron.com,phillip.allen@enron.com


In [9]:
# Insiders' emails and POIs' emails
insider_emails = df.email_address.replace('NaN', np.nan, regex=True).tolist()
poi_emails = df[df.POI == 'Yes'].email_address.tolist()

In [10]:
# Count the number of emails received and sent by POI
received_from_all = []
received_from_insider = []
received_from_poi = []
sent_to_all = []
sent_to_insider = []
sent_to_poi = []

for email in insider_emails:
    received_from_all.append(email_df[email_df.to == email].shape[0])
    received_from_insider.append(email_df[(email_df.from_.isin(insider_emails)) & \
                                          (email_df.to == email)].shape[0])
    received_from_poi.append(email_df[(email_df.from_.isin(poi_emails)) & \
                                  (email_df.to == email)].shape[0])
    sent_to_all.append(email_df[email_df.from_ == email].shape[0])
    sent_to_insider.append(email_df[(email_df.to.isin(insider_emails)) & \
                                  (email_df.from_ == email)].shape[0])
    sent_to_poi.append(email_df[(email_df.to.isin(poi_emails)) & \
                          (email_df.from_ == email)].shape[0])

df['from_all'] = received_from_all
df['to_all'] = sent_to_all
df['from_insider'] = received_from_insider
df['to_insider'] = sent_to_insider
df['from_poi'] = received_from_poi
df['to_poi'] = sent_to_poi

In [11]:
# Final dataset for analysis
df

,insider,salary,bonus,long_term_incentive,deferred_income,deferral_payments,loan_advances,other,expenses,director_fees,...,position,POI,bonus_salary_ratio,email_address,from_all,to_all,from_insider,to_insider,from_poi,to_poi
0,"ALLEN, PHILLIP K",201955.0,4175000.0,304805.0,-3081055.0,2869717.0,0.0,152.0,13868.0,0.0,...,NaN,No,20.672922,phillip.allen@enron.com,76,2099,13,83,0,24
1,"BADUM, JAMES P",0.0,0.0,0.0,0.0,178980.0,0.0,0.0,3486.0,0.0,...,NaN,No,0.000000,NaN,0,0,0,0,0,0
2,"BANNANTINE, JAMES M",477.0,0.0,0.0,-5104.0,0.0,0.0,864523.0,56301.0,0.0,...,NaN,No,0.000000,james.bannantine@enron.com,9,15,9,2,3,0
3,"BAXTER, JOHN C",267102.0,1200000.0,1586055.0,-1386055.0,1295738.0,0.0,2660303.0,11200.0,0.0,...,NaN,No,4.492666,NaN,0,0,0,0,0,0
4,"BAY, FRANKLIN R",239671.0,400000.0,0.0,-201641.0,260455.0,0.0,69.0,129142.0,0.0,...,NaN,No,1.668955,frank.bay@enron.com,4,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,"WINOKUR JR., HERBERT S",0.0,0.0,0.0,-25000.0,0.0,0.0,0.0,1413.0,108579.0,...,NaN,No,0.000000,NaN,0,0,0,0,0,0
140,"WODRASKA, JOHN",0.0,0.0,0.0,0.0,0.0,0.0,189583.0,0.0,0.0,...,NaN,No,0.000000,john.wodraska@enron.com,0,0,0,0,0,0
141,"WROBEL, BRUCE",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,No,0.000000,NaN,0,0,0,0,0,0
142,"YEAGER, F SCOTT",158403.0,0.0,0.0,0.0,0.0,0.0,147950.0,53947.0,0.0,...,Broadband strategist,Yes,0.000000,scott.yeager@enron.com,3,0,0,0,0,0


In [12]:
# Save into a csv file
path = os.getcwd() + '/enron.csv'
df.to_csv (path, index = False, header=True)